In [1]:
import numpy as np
import nnfs
from nnfs.datasets import spiral_data

nnfs.init()

class Layer_Dense:

    #layer initialization 
    def __init__(self, n_inputs, n_neurons):
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))

    def forward(self, inputs):
        #calculate output values form inputs, weights and biases
        self.output = np.dot(inputs, self.weights) + self.biases
class Activation_ReLU:

    def forward(self, inputs):
        self.output = np.maximum(0, inputs)

class Activation_Softmax:
    def forward(self, inputs):
        exp_values = np.exp(inputs - np.max(inputs, axis = 1, keepdims = True))

        #normalize them for each sample
        probabilities = exp_values / np.sum(exp_values, axis = 1, keepdims = True)

        self.output = probabilities

#common loss class
class loss:
    def calculate(self, output, y):
        sample_losses = self.forward(output,y)

        data_loss= np.mean(sample_losses)

        return data_loss

class Loss_CategoricalCrossentropy(loss):
    def forward(self, y_pred, y_true):
        samples = len(y_pred)
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)

        if len (y_true.shape) == 1:
            correct_confidences  = y_pred_clipped[
                range(samples),
                y_true
            ]
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(
                y_pred_clipped * y_true,
                axis = 1
            )
        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods

In [2]:
X, y = spiral_data(samples = 100, classes  =3)
dense1 = Layer_Dense(2,3)
activation1 = Activation_ReLU()
dense2 = Layer_Dense(3,3)
activation2 = Activation_Softmax()
loss_function = Loss_CategoricalCrossentropy()


In [3]:
lowest_loss = 9999999
best_dense1_weights = dense1.weights.copy()
best_dense1_biases = dense1.biases.copy()
best_dense2_weights = dense2.weights.copy()
best_dense2_biases = dense2.biases.copy()

In [4]:
for iteration in range(10000):
    dense1.weights = 0.05 * np.random.randn(2,3)
    dense1.biases = 0.05 * np.random.randn(1,3)
    dense2.weights = 0.05 * np.random.randn(3,3)
    dense2.biases = 0.05 * np.random.randn(1,3)

    dense1.forward(X)
    activation1.forward(dense1.output)
    dense2.forward(activation1.output)
    activation2.forward(dense2.output)

    loss = loss_function.calculate(activation2.output,y)

    predictions = np.argmax(activation2.output, axis = 1)
    accuracy = np.mean(predictions == y)
    

    #if loss is smaller - print and save weights and biases aside
    if loss < lowest_loss:
        print("new set of weights found, iteration:", iteration, "loss:", loss, "acc:", accuracy)

        best_dense1_weights = dense1.weights.copy()
        best_dense1_biases = dense1.biases.copy()
        best_dense2_weights = dense1.weights.copy()
        best_dense2_boases = dense1.biases.copy()
        lowest_loss = loss
    else:
        dense1.weights = best_dense1_weights.copy()
        dense1.biases = best_dense1_biases.copy()
        dense2.weights = best_dense2_weights.copy()
        dense2.biases = best_dense2_biases.copy()

new set of weights found, iteration: 0 loss: 1.1008568 acc: 0.3333333333333333
new set of weights found, iteration: 1 loss: 1.0990819 acc: 0.3333333333333333
new set of weights found, iteration: 3 loss: 1.098629 acc: 0.3333333333333333
new set of weights found, iteration: 11 loss: 1.0985013 acc: 0.3333333333333333
new set of weights found, iteration: 58 loss: 1.0984759 acc: 0.36666666666666664
new set of weights found, iteration: 87 loss: 1.0984341 acc: 0.3933333333333333
new set of weights found, iteration: 389 loss: 1.0983855 acc: 0.3333333333333333
new set of weights found, iteration: 602 loss: 1.0983611 acc: 0.3433333333333333
new set of weights found, iteration: 812 loss: 1.0983337 acc: 0.3333333333333333
new set of weights found, iteration: 1765 loss: 1.0982677 acc: 0.37
new set of weights found, iteration: 3766 loss: 1.0980942 acc: 0.3333333333333333
new set of weights found, iteration: 3979 loss: 1.0980837 acc: 0.3333333333333333
new set of weights found, iteration: 5582 loss: 